# Rendu Final Projet Fairness en IA

Étudiant 01 : MEDJADJ Mohamed Abderraouf <br>
Étudiant 02 : KERMADJ Zineddine <br>
Groupe : 01 <br>
Parcours : LDD3 Magistère d'Informatique

---

## I. Introduction

### a. Objectif du projet :
L’objectif de ce projet est d’**analyser un sous-ensemble de métadonnées et d’images** du NIH Chest X-ray Dataset, comprenant environ 5300 points de données, afin d’**identifier d’éventuels biais**. Après avoir appliqué une méthode de prétraitement pour **réduire ces biais et améliorer l’équité des données**, nous entraînerons un **modèle de classification d’images**. Enfin, nous analyserons **l’impact de la pondération sur les performances du modèle** ainsi que **l’effet du post-traitement** sur l’atténuation des biais.
### b. Description du dataset :
*Le NIH Chest X-ray Dataset est un vaste ensemble de données médicales comprenant **112 120 images** de radiographies thoraciques issues de **30 805 patients uniques**, avec des étiquettes de maladies générées par traitement automatique du langage naturel (**NLP**) à partir des rapports radiologiques. Ce jeu de données vise à pallier le **manque d’images médicales annotées**, un obstacle majeur au développement de systèmes de diagnostic assisté par ordinateur (CAD) cliniquement pertinents. Les étiquettes sont estimées à plus de **90 % de précision**, rendant cet ensemble adapté à l’apprentissage faiblement supervisé. Avant sa publication, le plus grand jeu de données disponible comptait seulement 4 143 images. Plus de détails sur l’ensemble de données et le processus d’annotation sont disponibles dans l’article en libre accès : « ChestX-ray8: Hospital-scale Chest X-ray Database and Benchmarks on Weakly-Supervised Classification and Localization of Common Thorax Diseases » (Wang et al.).*
### c. Contenu du dataset :
***Image Index** : Identifiant unique pour chaque image. <br>
**Finding Labels** : Diagnostiques associés à l'image (plusieurs diagnostics peuvent être présents). <br>
**Follow-up** # : Le numéro de suivi, indiquant si l'image appartient à un suivi ou à une première consultation. <br>
**Patient ID** : Identifiant unique pour chaque patient. <br>
**Patient Age** : L'âge du patient. <br>
**Patient Gender** : Le genre du patient. <br>
**View Position** : La position de l'image (par exemple, AP pour antéro-postérieur). <br>
**Dimensions et espacements de l'image** : Ces informations peuvent être utiles pour l'analyse des images, mais elles ne semblent pas directement liées à l'identification des biais.*

### SOMMAIRE:
<pre><b>
I.   Introduction
II.    0. Fonctions Utilitaires
II.    1. Préparation des données
II.    2. Analyse des données
II.    3. Identification des biais
III. Application des méthodes de preprocessing
IV.  Application des méthodes de postprocessing
V.   Analyse et compréhension
VI.  Conclusion
</b></pre>

---
# **REMARQUES IMPORTANTES:**
### *a. Les explications des sorties, commentaires des graphiques, etc, sont inclus dans des cellules de **code, et pas markdown**, veuillez donc s'il vous plaît ne pas ignorer les lignes commentées.*

### *b. Pour l'entrainement, veuillez consulter "training.ipynb".*
---

## II.0. Fonctions utilitaires

### a. Import des librairies nécessaires

In [ ]:
import os
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.metrics import confusion_matrix
from aif360.datasets import BinaryLabelDataset
from aif360.sklearn.metrics import *
from aif360.algorithms.preprocessing import *
from train_classifieur import train_classifier, pred_classifier
from scipy.sparse import csr_matrix
import random
from IPython.display import Image, display
from PIL import Image
from IPython.display import Image as IPImage, display
from aif360.algorithms.postprocessing.calibrated_eq_odds_postprocessing import CalibratedEqOddsPostprocessing
import matplotlib.pyplot as plt
import seaborn as sns
from aif360.metrics import ClassificationMetric
from aif360.datasets import BinaryLabelDataset
from aif360.algorithms.postprocessing import RejectOptionClassification

### b. Les fonctions utilitaires

In [ ]:
# Fonction pour le calcul des métriques de fairness

def get_group_metrics(
    y_true,
    y_pred=None,
    prot_attr=None,
    priv_group=1,
    pos_label=1,
    sample_weight=None,
):
    group_metrics = {}
    group_metrics["base_rate"] = base_rate(
        y_true=y_true, pos_label=pos_label, sample_weight=sample_weight
    )
    group_metrics["statistical_parity_difference"] = statistical_parity_difference(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
    )
    group_metrics["disparate_impact_ratio"] = disparate_impact_ratio(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
    )
    if not y_pred is None:
        group_metrics["equal_opportunity_difference"] = equal_opportunity_difference(
            y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["average_odds_difference"] = average_odds_difference(
            y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["conditional_demographic_disparity"] = conditional_demographic_disparity(
            y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["smoothed_edf"] = smoothed_edf(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["df_bias_amplification"] = df_bias_amplification(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, pos_label=pos_label, sample_weight=sample_weight
        )
    return group_metrics

In [ ]:
# Fonction pour tracer des matrices de confusion

def plot_confusion_matrix(y_true, y_pred, labels=["sain", "malade"], normalize=False, title="Matrice de confusion"):
    cm = confusion_matrix(y_true, y_pred)
    
    if normalize:
        cm = cm.astype('float') / len(y_true) * 100
    
    cm_df = pd.DataFrame(cm, index=labels, columns=labels)
    
    fig = px.imshow(cm_df, 
                    labels=dict(x="Prédiction", y="Vérité", color="Fréquence (%)" if normalize else "Fréquence"), 
                    x=labels, 
                    y=labels, 
                    color_continuous_scale='Blues',
                    range_color=[0, 100] if normalize else None) 
    
    for i in range(len(cm_df)):
        for j in range(len(cm_df.columns)):
            fig.add_annotation(
                x=j,
                y=i,
                text=f'{cm_df.iloc[i, j]:.2f}%' if normalize else f'{cm_df.iloc[i, j]}',
                showarrow=False,
                font=dict(color="black", size=14),
                align="center"
            )
    
    fig.update_layout(title=title, xaxis_title="Prédiction", yaxis_title="Vérité")
    fig.show()

In [ ]:
# Fonction pour afficher des matrices de confusion séparées pour chaque groupe défini par group_columns
def plot_confusion_matrix_by_group(y_true, y_pred, df, group_columns, labels=None, normalize=False):
    for group_value, group_df in df.groupby(group_columns):
        y_true_group = y_true[group_df.index]
        y_pred_group = y_pred[group_df.index]
        
        print(f"Matrice de confusion pour {group_columns}: {group_value}")
        plot_confusion_matrix(y_true_group, y_pred_group, labels, normalize, title=f"Matrice de Confusion ({group_columns}={group_value})")


## II.1. Preparation des données

### a. Chargement du dataset

In [ ]:
DATA_DIR = "./DATA"
PREDS_DIR = "./expe_log"
df = pd.read_csv(PREDS_DIR+"/preds.csv")

### b. Exploration préliminaire

In [ ]:
print(df.shape)
# Affichons les 5 premiers points de donnée du dataset
df.head()

In [ ]:
# Vérifions les types de données et les valeurs manquantes
df.info()

# Explication des sorties:
# Le dataset est composé de 12 colonnes, dont 8 correspondent à des features numériques, et 4 catégorielles.
# Toutes les colonnes ne contiennent pas de valeurs nulles sauf la dernière (Unnamed: 11), qui contient que des valeurs nulles.
# Le dataset contient 54009 points de données.

In [ ]:
# Statistiques descriptives
df.describe()

# Explication des sorties:
# Max: On remarque qu'il y a une valeur max = 412 pour l'age, qui n'est pas normal (outlier), et qui peut être dû à une erreur de frappe.
# Count = nombre de points de données sauf pour 'Unnamed: 11', cette colonne contient que des null.
# Mean: Moyenne des valeurs par colonne, pas de remarque importante.
# Std: Standard deviation des valeurs par colonne, pas de remarque importante.
# Min: Le min des valeurs par colonne, pas de remarque importante.
# Les Quantiles: pas de remarque importante.

### c. Préparation du dataset

In [ ]:
# On sauvegarde le dataframe original avant toute transformation
original_df = df.copy()

In [ ]:
# Fonction pour faire des train-test split:

def train_test_split(df):
    train_sain_path = DATA_DIR+"/train/sain"
    train_malade_path = DATA_DIR+"/train/malade"

    train_images = set(os.listdir(train_sain_path) + os.listdir(train_malade_path))

    df["in_train"] = df["Image Index"].apply(lambda x: 1 if x in train_images else 0)

In [ ]:
# Élimination des outliers (age > 130 years old)
df = df[df['Patient Age'] <= 130]

# Séparation des données train-test
train_test_split(df)

# Séparation des colonnes liées aux images des autres métadonnées
df_image_related = df[['Image Index', 'OriginalImage[Width', 'Height]', 'OriginalImagePixelSpacing[x', 'y]', 'View Position']]
df_others = df.drop(columns=['Image Index', 
    'OriginalImage[Width', 'Height]', 'OriginalImagePixelSpacing[x', 'y]', 'View Position'])

# Encodage de 'preds' et 'labels' en binaire
df_others["preds"] = df_others["preds"].map({"sain": 0, "malade": 1}) 
df_others["labels"] = df_others["labels"].map({"sain": 0, "malade": 1})

# Déplacement du label 'Finding Labels' à la fin du dataframe.
df_others = df_others[[col for col in df_others.columns if col != 'Finding Labels'] + ['Finding Labels']]
df_original_label = df_others[['Finding Labels']]

print(df_others.shape)
# On remarque que le nombre de points de données a diminué de 6 (outliers eliminés).

df_others.head()

In [ ]:
# VERSION 1: encodage one-hot du label

# a. Création d'une colonne 'Finding Labels' contenant une liste des labels
df_encoded_OH = df_others.copy()
df_encoded_OH['Finding Labels'] = df_encoded_OH['Finding Labels'].replace('No Finding', '').str.split('|')
df_encoded_OH['Finding Labels'] = df_encoded_OH['Finding Labels'].apply(lambda x: [] if x == [''] else x)

# b. Encodage des colonnes en One-Hot avec le prefix "Finding_"
all_labels = set([label for sublist in df_encoded_OH['Finding Labels'] for label in sublist])
for label in all_labels:
    column_name = f"Finding_{label.lower().replace(' ', '_')}"
    df_encoded_OH[column_name] = df_encoded_OH['Finding Labels'].apply(lambda x: 1 if label in x else 0)
df_encoded_OH = df_encoded_OH.drop(columns=['Finding Labels'])

df_encoded_OH

In [ ]:
# VERSION 2: encodage binaire (1 ssi patient malade) du label

df_encoded_bool = df_others.copy()
df_encoded_bool = df_encoded_bool.drop(columns=['Finding Labels'])

df_encoded_bool

## II.2. Analyse des données

### 0. Séparation du label, des features numériques et catégorielles

In [ ]:
df_encoded = df_encoded_OH
df_encoded.dtypes
# Toutes les features sont numériques, sauf le genre (sexe)

In [ ]:
# Séparation du label, des features numériques et catégorielles
one_hot_label = [col for col in df_encoded.columns if col.startswith("Finding")] # pour la version 1 de l'encodage
boolean_label = 'labels' # pour la version 2 de l'encodage

numerical_features = list(set(df_encoded.select_dtypes(include=np.number).columns) - set(one_hot_label))
categorical_features = list(set(df_encoded.columns) - set(numerical_features) - set(one_hot_label))

### Analyse univariée

### a. Analyse de la corrélation (Feature-Label et Feature-Feature)

In [ ]:
# Fonction qui calcule la corrélation linéaire entre deux features
def compute_correlation(df, cola, colb):
  return np.corrcoef(df[cola].values, df[colb].values)[0][1]

In [ ]:
# Calcul des corrélations entre les features et les colonnes du label (one-hot) (Feature-Label).
for num_feature in numerical_features:
  if num_feature not in {"labels", "preds", "in_train"}:
    for label_col in one_hot_label:
      corr = compute_correlation(df_encoded, label_col, num_feature)
      if np.abs(corr) > 0.05 :
        print(num_feature, label_col, corr)

## On n'affiche que les couples de features qui ont une corrélation (valeur absolue) supérieure à 0.05
## Comme on le voit, il s'agit de très faibles corrélations (pas de biais linéaire)

In [ ]:
# Calcul des corrélations entre les features et les colonnes du label (booléen) (Feature-Label).
for num_feature in numerical_features:
    if num_feature not in {"labels", "preds", "in_train"}:
        corr = compute_correlation(df_encoded_bool, boolean_label, num_feature)
        if np.abs(corr) > 0.05 :
            print(num_feature, boolean_label, corr)

## On n'affiche que les couples de features qui ont une corrélation (valeur absolue) supérieure à 0.05
## Comme on le voit, il s'agit de très faibles corrélations (pas de biais linéaire)

In [ ]:
# Calcul des corrélations entre les features (Feature-Feature).
for num_feature in numerical_features:
  if num_feature not in {"labels", "preds", "in_train"}:
    for num_feature2 in (set(numerical_features)-set([num_feature])):
      if num_feature2 not in {"labels", "preds", "in_train"}:
        corr = compute_correlation(df_encoded, num_feature, num_feature2)
        if np.abs(corr) > 0.05 :
          print(num_feature, num_feature2, corr)

## On n'affiche que les couples de features qui ont une corrélation (valeur absolue) supérieure à 0.05
## Comme on le voit, il s'agit de très faibles corrélations (pas de biais linéaire)

### b. Analyse des biais liés au sexe

In [ ]:
fig = px.pie(df_encoded, names="Patient Gender", title="Distribution du sexe")
fig.show()

# Explication des sorties:
# On remarque que le nombre de points de données 'masculin' est plus élevé que celui des 'féminin', 
# ce qui pourrait indiquer une sous-représentation des femmes dans le dataset.

In [ ]:
fig = px.histogram(df_encoded_bool, x='Patient Gender', color=boolean_label, \
                  title=f"{'Patient Gender'} distribution by {boolean_label}")
fig.show()

# Explication des sorties:
# On remarque que la distribution du sexe selon 'is_ill' est balancée (50% malade, 50% non malade, dans les deux sexes).
# Donc on est bon vis-à-vis de ça.

In [ ]:
gender_col_bool = (df_encoded_bool['Patient Gender'] == 'M') # Conversion {'F' -> 0, 'M' -> 1}

def calc_dir_gender(df, label):
    dir = disparate_impact_ratio(
        y_true=df[label],
        prot_attr=gender_col_bool,
        pos_label=0
    )
    return dir

print("global_is_ill", calc_dir_gender(df_encoded_bool, boolean_label))

# Explication des sorties:
# DIR ~ 1, donc pas de biais pas de classe (M/F) favorisée vis-à-vis le label 'is_ill'.

In [ ]:
def display_categorical_hist(df, cat_feature, target):
  fig = px.histogram(df, x=cat_feature, color=target)
  fig.show()

def display_categorical_hist_percent(df, cat_feature, target):
  if cat_feature != "labels" and cat_feature != "preds" :
    df_summarized = df.groupby([target,cat_feature]).agg("count").reset_index()
    df_summarized[f"percent of {cat_feature}"] = df_summarized[[cat_feature,"Patient Age"]].apply(
      lambda x: 100*x["Patient Age"]/df_summarized[df_summarized[cat_feature]==x[cat_feature]]["Patient Age"].sum(), axis=1
    )
    df_summarized[target] = df_summarized[target].astype(str)
    fig = px.bar(df_summarized, x=f"{cat_feature}", y=f"percent of {cat_feature}", color=target)
    fig.show()

In [ ]:
for cat_feature in categorical_features:
  for label_col in one_hot_label:
    display_categorical_hist_percent(df_encoded, cat_feature, label_col)

# Explication des sorties:
# En général, il n'y a pas de classe favorisée, la distribution de chaque pathologie est similaire sur les deux sexes.
# Mais on remarque qu'il y a des maladies sous-représentées, confirmons cela avec la visualisation suivante (partie 3.c).

In [ ]:
for label_col in one_hot_label:
    dir = calc_dir_gender(df_encoded, label_col)
    print(label_col, dir)

# Explication des sorties:
# Tous les DIRs calculés sont ~ 1, donc pas de biais pas de classe (M/F) favorisée vis-à-vis les labels one-hot.

### c. Analyse des biais liés à la distribution des labels (one-hot)

In [ ]:
category_counts = df_encoded[one_hot_label].sum()
fig = px.bar(
    x=category_counts.index,
    y=category_counts.values,
    labels={'x': 'Disease Type', 'y': 'Count of 1s'},
    title='Distribution of One-Hot Encoded Labels'
)
fig.show()

# Explication des sorties:
# On remarque une distribution non équilibrée sur les différentes maladies, ce qui peut induire un modèle biaisé.
# Solution: introduire des weights, détaillons ça après.

### d. Analyse des biais liés à l'age

In [ ]:
fig = px.box(df_encoded_bool, x=boolean_label, y='Patient Age', title=f"age distribution by {boolean_label}")
fig.show()

fig = px.histogram(df_encoded_bool, x='Patient Age', color=boolean_label, barmode="overlay", \
                  title=f"age distribution by {boolean_label}")
fig.show()

# Explication des sorties:
# On remarque que 50% des patients qui sont condensés au mileu ~[30ans -> 65ans].
# Les distributions d'age sur les deux classes (malade/non malade) sont similaires.
# Mais on pourrait argumenter que les catégories d'age ~[0 - 30 ans] et ~[65+ ans], sont sous-représentés.
# Pour le confirmer, passons au graphique suivant (Discretisation d'age).

In [ ]:
# Discretisation de l'age
discrete_age = {}
discrete_age['age_group'] = pd.cut(df_encoded_bool['Patient Age'], bins=[0, 30, 65, 130], labels=['0-30', '30-65', '65+'], right=True)
df_encoded_bool['age_group'] = discrete_age['age_group']
fig = px.histogram(discrete_age, x='age_group', title='Distribution des classes d\'age',
                   labels={'age_group': 'Classes d\'age'}, 
                   text_auto=True)
fig.show()

# Explication des sorties:
# Voilà, on voit bien le deséquilibre dans la distribution des classes d'age, les classes '0-30' et '65+' sont sous-représentées
# en comparaison avec '30-65', d'où la présence d'un potentiel biais lié à ça.

In [ ]:
# Determinons le unpriviliged group



In [ ]:
# On encode les classes d'age en booléen comme suit:

df_encoded_bool["age_30_65"] = (df_encoded_bool['age_group'] == '30-65').astype(int)
df_encoded_bool = df_encoded_bool.drop(columns=['age_group'])
df_encoded_bool

### e. Analyse des biais liés au "Patient ID"

In [ ]:
fig = px.box(df_encoded_bool, x=boolean_label, y='Patient ID', title=f"patient_id distribution by {boolean_label}")
fig.show()

fig = px.histogram(df_encoded_bool, x='Patient ID', color=boolean_label, \
                  title=f"patient_id distribution by {boolean_label}")
fig.show()

# Explication des sorties:
# On peut dire de la première figure que la distribution des patient_id est similaire sur les deux classes (malade/non malade).
# On peut même dire que la distribution est pseudo-uniforme.
# Mais en regardant le deuxième graphique, on remarque une petite anomalie au voisinage de l'id 25000, 
# et qui peut biaiser le modèle.

### Analyse bivariée

### f. Analyse (bivariée) des biais liés au 'Follow-up'

In [ ]:
# Fig 01
fig = px.histogram(df_encoded, x="Patient ID", y="Follow-up #", title="Distribution du nombre de suivis")
fig.show()

# Fig 02
fig = px.histogram(df_encoded, x="Patient Gender", y="Follow-up #", title="Nombre de suivis par genre")
fig.show()

# Fig 03
fig = px.histogram(df_encoded, x="Patient Age", y="Follow-up #", title="Relation entre l’âge et le nombre de suivis")
fig.show()

# Fig 03
df_melted = df_encoded.melt(id_vars=["Follow-up #"], value_vars=[col for col in df_encoded.columns if "Finding_" in col], 
                     var_name="disease", value_name="presence")
df_melted = df_melted[df_melted["presence"] == 1]

# Fig 04
fig = px.histogram(df_melted, x="disease", y="Follow-up #", title="Nombre de suivis par maladie")
fig.show()


# Explication des sorties:
# Fig 01: Deux petites anomalies au voisinage des IDs 10000 (pic très marqué du nombre de suivis) et 25000 (forte baisse, presque à zéro), 
# cela peut biaiser le modèle quand on prédit des valeurs à ces voisinages.
# Fig 02: Même remarque que dans l'étude des biais liés au sexe (b).
# Fig 03: Même remarque que dans l'étude des biais liés à l'age (d).
# Fig 04: Même remarque que dans l'étude des biais liés à la distribution des labels one-hot (c).

In [ ]:
finding_columns = [col for col in df_encoded.columns if col.startswith("Finding")]

sparse_matrix = csr_matrix(df_encoded[finding_columns].values)

co_occurrence_matrix = sparse_matrix.T @ sparse_matrix

total_occurrences = np.array(sparse_matrix.sum(axis=0)).flatten()
probability_matrix = co_occurrence_matrix.toarray() / total_occurrences

co_occurrence_df = pd.DataFrame(co_occurrence_matrix.toarray(), index=finding_columns, columns=finding_columns)
probability_df = pd.DataFrame(probability_matrix, index=finding_columns, columns=finding_columns)

def display_matrix(df, title):
    return df.style.background_gradient(cmap="Blues", axis=None).set_caption(title)

styled_co_occurrence_df = display_matrix(co_occurrence_df, "Matrice de Co-Occurrence")
styled_probability_df = display_matrix(probability_df, "Matrice de Probabilités Conditionnelles")

print("Matrice de co-occurrence:")
display(styled_co_occurrence_df)

print("\nMatrice de probabilités conditionnelles:")
display(styled_probability_df)


### Analyse d'images

La principale nouveauté depuis le mi-projet est l’intégration d’images dans le dataset. Cependant, ces nouvelles données peuvent introduire de nouveaux biais, tels qu’une faible luminosité, la présence de dispositifs médicaux intra-corporels, ou d'autres artefacts visuels susceptibles d’influencer l’analyse.

In [ ]:
import os
import random
from IPython.display import Image, display

# Liste de chemins vers tes dossiers (paths)
paths = [
    "DATA/train/sain",
    "DATA/train/malade",
    "DATA/valid/sain",
    "DATA/valid/malade",
]

# Récupère tous les fichiers .png dans les dossiers
toutes_les_images = []
for path in paths:
    if os.path.exists(path):
        images = [os.path.join(path, f) for f in os.listdir(path) if f.lower().endswith('.png')]
        toutes_les_images.extend(images)

# Choisir un certain nombre d'images aléatoires (par exemple 6)
nombre_images = 6
images_choisies = random.sample(toutes_les_images, min(nombre_images, len(toutes_les_images)))

# Affiche les images dans le notebook
for chemin in images_choisies:
    display(Image(filename=chemin))


In [ ]:
from PIL import Image
from IPython.display import Image as IPImage, display

# Fonction pour calculer la luminosité moyenne d'une image
def calculer_luminosite(image_path):
    # Ouvrir l'image
    img = Image.open(image_path)
    
    # Convertir l'image en niveaux de gris (pour analyser la luminosité)
    grayscale_img = img.convert("L")
    
    # Convertir l'image en un tableau numpy
    img_array = np.array(grayscale_img)
    
    # Calculer la luminosité moyenne de l'image
    luminosite_moyenne = np.mean(img_array)
    
    return luminosite_moyenne

# Fonction pour récupérer toutes les images dans un répertoire
def recuperer_images_du_repertoire(dossier):
    images = []
    for file in os.listdir(dossier):
        if file.lower().endswith('.png'):  # Seulement les fichiers .png
            images.append(os.path.join(dossier, file))
    return images

# Variable pour stocker les images à faible et forte luminosité
images_faible_luminosite = []
images_forte_luminosite = []

# Pour chaque dossier, récupérer toutes les images et choisir celle ayant la plus faible luminosité
for path in paths:
    if os.path.exists(path):
        # Récupérer toutes les images du dossier
        images = recuperer_images_du_repertoire(path)
        
        if images:
            # Calculer la luminosité pour chaque image
            luminosites = [(image, calculer_luminosite(image)) for image in images]
            
            # Trouver l'image avec la luminosité la plus faible
            image_min_luminosite = min(luminosites, key=lambda x: x[1])
            image_max_luminosite = max(luminosites, key=lambda x: x[1])
            
            # Ajouter l'image à faible et forte luminosité à la liste
            images_faible_luminosite.append(image_min_luminosite[0])
            images_forte_luminosite.append(image_max_luminosite[0])

# Affichage des images à faible luminosité
for chemin in images_faible_luminosite:
    display(IPImage(filename=chemin))
    
for chemin in images_forte_luminosite:
    display(IPImage(filename=chemin))


In [ ]:
def foreground_filter(gris, threshold):
    """
    Retourne un masque booléen indiquant quels pixels de l'image en niveaux de gris (gris)
    ont une valeur inférieure au seuil.
    """
    return gris < threshold

def lightness(img: Image.Image) -> float:
    """
    Calcule la "faible luminosité" d'une image PIL.
    L'image est convertie en niveaux de gris par la moyenne des canaux R, G et B.
    La fonction retourne la moyenne des pixels du premier plan (où la luminosité est inférieure au seuil).
    """
    M = np.array(img)
    
    gris = np.mean(M, axis=2)
    
    F = foreground_filter(gris, 130)
    
    if np.any(F):
        return np.mean(gris[F])
    else:
        return np.mean(gris)

paths = [
    "DATA/train/sain",
    "DATA/train/malade",
    "DATA/valid/sain",
    "DATA/valid/malade",
]

data = []

for dossier in paths:
    if os.path.exists(dossier):
        image_files = [os.path.join(dossier, f) for f in os.listdir(dossier) if f.lower().endswith('.png')]
        
        for image_path in image_files:
            with Image.open(image_path) as img:
                lum = lightness(img)
            data.append({
                "Image Index": os.path.basename(image_path),
                "Folder": dossier,
                "lightness": lum
            })

df = pd.DataFrame(data)
df.to_csv("lightness.csv", index=False)
df.head()

### e. Analyse des biais liés à la luminosité des images

In [ ]:
# Vérifions d'abord si la colonne 'lightness' existe dans le DataFrame
if 'lightness' in df.columns:
    plt.figure(figsize=(10, 6))
    sns.histplot(df['lightness'], kde=True, bins=30, color='blue')
    plt.title('Distribution de la Luminosité (Lightness)', fontsize=16)
    plt.xlabel('Lightness', fontsize=14)
    plt.ylabel('Fréquence', fontsize=14)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()
else:
    print("La colonne 'lightness' n'existe pas dans le DataFrame.")

In [ ]:
# Vérifions d'abord si la colonne 'lightness' existe dans le DataFrame
if 'lightness' in df.columns and 'Folder' in df.columns:
    df['labels'] = df['Folder'].apply(lambda x: 1 if 'malade' in x else 0)
    plt.figure(figsize=(10, 6))
    sns.kdeplot(data=df, x='lightness', hue='labels', fill=True, common_norm=False, 
                palette={0: 'green', 1: 'red'}, alpha=0.5)
    plt.title('Distribution de la Luminosité (Lightness) par État du Patient', fontsize=16)
    plt.xlabel('Lightness', fontsize=14)
    plt.ylabel('Densité', fontsize=14)
    plt.legend(title='État du Patient', labels=['Sain', 'Malade'])
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()
else:
    print("Les colonnes 'lightness' ou 'Folder' n'existent pas dans le DataFrame.")

Les distributions de la variable « lightness » pour les malades et les sains se chevauchent largement, ce qui indique qu'il n'existe pas de séparation nette entre les deux groupes. Cela suggère que la variation de la luminosité observée n'est pas le reflet d'un biais discriminatoire dans le modèle, mais plutôt d'une fluctuation technique inhérente aux réglages d'exposition ou à la standardisation des acquisitions radiographiques. En conséquence, on peut considérer que cette variable, en l'état, n'induit pas de biais dans la prise de décision, bien qu'il soit toujours recommandé de procéder à des analyses statistiques complémentaires pour confirmer que cette caractéristique n'influence pas indûment les prédictions.

### Préparation du dataset AIF360

In [ ]:
# Encodage sous format AIF360

protected_attributes = ['Patient Gender', 'age_30_65']
protected_attribute = protected_attributes[1]

def get_aif360_data(df):
    dff = df.copy()
    dff["Patient Gender"] = dff["Patient Gender"].map({"M": 0, "F": 1})

    ret_df = BinaryLabelDataset(
        favorable_label=0,  # "sain" est la classe favorable
        unfavorable_label=1,  # "malade" est la classe défavorable
        df=dff,
        label_names=['labels'], 
        protected_attribute_names=protected_attributes
    )

    ret_df_train = BinaryLabelDataset(
        favorable_label=0,  # "sain" est la classe favorable
        unfavorable_label=1,  # "malade" est la classe défavorable
        df=dff[dff["in_train"]==1],
        label_names=['labels'], 
        protected_attribute_names=protected_attributes
    )

    ret_df_valid = BinaryLabelDataset(
        favorable_label=0,  # "sain" est la classe favorable
        unfavorable_label=1,  # "malade" est la classe défavorable
        df=dff[dff["in_train"]==0],
        label_names=['labels'], 
        protected_attribute_names=protected_attributes
    )

    return ret_df, ret_df_train, ret_df_valid

aif_df, aif_df_train, aif_df_valid = get_aif360_data(df_encoded_bool)

aif_df

In [ ]:
def print_metrics(labels, preds=None, gender_or_age=0):
    if gender_or_age == 0:
        print("PROTECTED: Patient Gender")
        metrics_before_training = get_group_metrics(
            y_true=labels,
            y_pred=preds,
            prot_attr=aif_df_valid.protected_attributes[:, 0],
            priv_group=1,
            pos_label=0
        )

        # Affichage des résultats
        for metric, value in metrics_before_training.items():
            print(f"{metric}: {value:.4f}")
    else:
        print("PROTECTED: age_30_65")
        metrics_before_training = get_group_metrics(
            y_true=labels,
            y_pred=preds,
            prot_attr=aif_df_valid.protected_attributes[:, 1],
            priv_group=1,
            pos_label=0
        )

        # Affichage des résultats
        for metric, value in metrics_before_training.items():
            print(f"{metric}: {value:.4f}")

In [ ]:
def get_test_Ys(csv_path):
    df = pd.read_csv(csv_path)
    train_test_split(df)
    df["preds"] = df["preds"].map({"sain": 0, "malade": 1}) 
    df["labels"] = df["labels"].map({"sain": 0, "malade": 1})
    test_df = df[df["in_train"] == 0]
    preds = test_df["preds"]
    labels= test_df["labels"]
    return labels, preds

labels, preds = get_test_Ys(PREDS_DIR+"/preds.csv")

print_metrics(labels, gender_or_age=0)
print("====================")
print_metrics(labels, gender_or_age=1)

# Explication des sorties:




In [ ]:
print_metrics(labels, preds, gender_or_age=0)
print("====================")
print_metrics(labels, preds, gender_or_age=1)

# Explication des sorties:


# -> On se concentre plus sur l'age

In [ ]:
y_true = df_encoded_bool["labels"]
y_pred = df_encoded_bool["preds"]
plot_confusion_matrix(y_true, y_pred, normalize=True)

# Explication des sorties:




In [ ]:
plot_confusion_matrix_by_group(y_true, y_pred, df_encoded_bool, group_columns=["age_30_65"], labels=["sain", "malade"], normalize=True)

# Explication des sorties:




## II.3. Identification des biais

#### Résumé des biais identifiés précédemment:
**a. Une sous-représentation des femmes dans le dataset:**
*42% de femmes versus 58% d'hommes, n'est pas une grande différence, de plus, le disparate impact ratio (DIR)
est très proche de 1 dans tous les cas, ce qui indique qu'il n’y a pas de biais significatif en termes de différence de traitement entre les hommes et les femmes.*

**b. Une sous-représentation de quelques maladies dans le dataset (deséquilibre label):**
*On remarque que quelques maladies sont sous-représentées (exemple: Hernia, Fibrosis, ...), d'autres sont sur-représentées (exemple: Infiltration, Iffusion, ...), cela peut entraîner un biais, car le modèle peut par exemple toujours prédire qu'un patient n'a pas Hernia (par exemple), sans être vraiment pénalisé, et donc ça peut donner de faux scores élevés.*

**c. Une sous-représentation de quelques classes d'age:**
*De même, dû à la sous-représentation des classes d'âge jeunes et vieilles, un modèle entrainé sur ce dataset peut donner des prédictions meilleures pour la classe du mileu [30-65 ans].*

**d. Des anomalies liées à la distribution de patient_id:**
*L'anomalie qu'on a dans la distribution des Patient IDs au voisinage de l'ID 25000 (probablement dûe à des données manquantes), peut donner un modèle qui répond toujours par 'is_ill = 0' (Patient n'est pas malade), si l'ID du patient est dans le voisinage de 25000, qui pose un problème.*

---

## III. Application des méthodes de preprocessing

### a. Reweighing

In [ ]:
# On applique le Reweighing sur le dataset
RW = Reweighing(unprivileged_groups=[{protected_attribute: 0}], privileged_groups=[{protected_attribute: 1}])
RW.fit(aif_df_train)
transformed_dataset_RW = RW.transform(aif_df)

df_encoded_bool["WEIGHTS"] = transformed_dataset_RW.instance_weights

# On reconcatène les colonnes relatives aux images
df_concatenated = pd.concat([df_image_related, df_encoded_bool], axis=1)

# On écrit les poids dans les metadatas
path_to_csv_RW = DATA_DIR+"/metadata_RW.csv"
df_concatenated.to_csv(path_to_csv_RW, index=False)


# On entraine le modele sur ces données (voir training.ipynb) -> resultat: preds_RW.csv


# On charge les prédictions
path_to_preds_RW = PREDS_DIR+"/preds_RW.csv"
labels, preds = get_test_Ys(path_to_preds_RW)

print_metrics(labels, preds, gender_or_age=0)
print("====================")
print_metrics(labels, preds, gender_or_age=1)

# Explication des sorties:




### b. DisparateImpactRemover

In [ ]:
# On applique le DIR sur le dataset
DIR = DisparateImpactRemover(sensitive_attribute=protected_attribute,repair_level=1.0)
transformed_dataset_DIR = DIR.fit_transform(aif_df)

# On reconcatène les colonnes relatives aux images
df_concatenated = pd.concat([df_image_related.reset_index(drop=True), 
    transformed_dataset_DIR.convert_to_dataframe()[0].reset_index(drop=True)], axis=1)

# On écrit les poids dans les metadatas
path_to_csv_DIR = DATA_DIR+"/metadata_DIR.csv"
df_concatenated.to_csv(path_to_csv_DIR, index=False)


# On entraine le modele sur ces données (voir training.ipynb) -> resultat: preds_DIR.csv


# On charge les prédictions
path_to_preds_DIR = PREDS_DIR+"/preds_DIR.csv"
labels, preds = get_test_Ys(path_to_preds_DIR)

print_metrics(labels, preds, gender_or_age=0)
print("====================")
print_metrics(labels, preds, gender_or_age=1)

# Explication des sorties:




### c. LFR

In [ ]:
# On applique le LFR sur le dataset
LFR = LFR(
    unprivileged_groups=[{protected_attribute: 0}],
    privileged_groups=[{protected_attribute: 1}],
    k=5,
    Ax=0.01,
    Ay=1.0,
    Az=50.0,
    print_interval=250,
    verbose=1,
    seed=None,
)
LFR.fit(aif_df_train, maxiter=5000, maxfun=5000)
transformed_dataset = LFR.transform(aif_df)

# On reconcatène les colonnes relatives aux images
df_concatenated = pd.concat([df_image_related.reset_index(drop=True), 
    transformed_dataset_DIR.convert_to_dataframe()[0].reset_index(drop=True)], axis=1)

# On écrit les poids dans les metadatas
path_to_csv_LFR = DATA_DIR+"/metadata_LFR.csv"
df_concatenated.to_csv(path_to_csv_LFR, index=False)


# On entraine le modele sur ces données (voir training.ipynb) -> resultat: preds_DIR.csv


# On charge les prédictions
path_to_preds_LFR = PREDS_DIR+"/preds_LFR.csv"
labels, preds = get_test_Ys(path_to_preds_LFR)

print_metrics(labels, preds, gender_or_age=0)
print("====================")
print_metrics(labels, preds, gender_or_age=1)

# Explication des sorties:




---

## IV. Application des méthodes de postprocessing

In [ ]:
# Définir les chemins vers les dossiers d'entraînement
path_train_sain = DATA_DIR + "/train/sain"
path_train_malade = DATA_DIR + "/train/malade"

# Lister les noms de fichiers d’images présents dans les deux sous-dossiers d’entraînement
images_entraînement  = set(os.listdir(path_train_sain) + os.listdir(path_train_malade))


def convert_all_to_numericals(df):
    # Nettoyer les noms de colonnes
    df.columns = df.columns.str.strip()
    
    # Ajouter une colonne "in_train" si elle n'existe pas encore
    if "in_train" not in df.columns:
        df["in_train"] = df["Image Index"].apply(lambda x: 1 if x in images_entraînement else 0)
    
    # Convertir les prédictions en 0/1 si ce n'est pas déjà numérique
    if not pd.api.types.is_numeric_dtype(df["preds"]):
        df["preds"] = df["preds"].map({"sain": 0, "malade": 1})
    
    # Convertir les étiquettes en 0/1 si nécessaire
    if not pd.api.types.is_numeric_dtype(df["labels"]):
        df["labels"] = df["labels"].map({"sain": 0, "malade": 1})
    
    # Convertir le genre en 0/1 (0 = M, 1 = F)
    if not pd.api.types.is_numeric_dtype(df["Patient Gender"]):
        df["Patient Gender"] = df["Patient Gender"].map({"M": 0, "F": 1})
    
    # Convertir la position de vue si elle est présente et non numérique
    if "View Position" in df.columns and not pd.api.types.is_numeric_dtype(df["View Position"]):
        df["View Position"] = df["View Position"].map({"AP": 0, "PA": 1})
    
    # Ajouter une colonne "+40ans" indiquant si l'âge du patient est supérieur à 40
    if "+40ans" not in df.columns:
        df["+40ans"] = (df["Patient Age"] > 40).astype(int)
    
    return df


In [ ]:
#On numérise toutes les colonnes
preddf = df_encoded_bool.copy()
preddf = convert_all_to_numericals(preddf)


protected_attributes = ['Patient Gender', '+40ans']

protected_attribute = protected_attributes[1]
priviliged_group = 0
unpriviliged_group = 1

if protected_attribute in list(preddf.columns):
    dataset = BinaryLabelDataset(
        favorable_label=1,  # "Sain" est la classe favorable
        unfavorable_label=0,  # "Malade" est la classe défavorable
        df=preddf,
        label_names=["labels"],
        protected_attribute_names=[protected_attribute]
    )

    t = preddf[preddf["in_train"]==1].copy().reset_index()
    train_dataset = BinaryLabelDataset(
        favorable_label=1,  # "Sain" est la classe favorable
        unfavorable_label=0,  # "Malade" est la classe défavorable
        df=t,
        label_names=["labels"],
        protected_attribute_names=[protected_attribute]
    )


    d = preddf[preddf["in_train"]==0].copy().reset_index()
    test_dataset = BinaryLabelDataset(
        favorable_label=1,  # "Sain" est la classe favorable
        unfavorable_label=0,  # "Malade" est la classe défavorable
        df=d,
        label_names=["labels"],
        protected_attribute_names=[protected_attribute]
    )

else:
    print(f"Colonne protégée '{protected_attribute}' non trouvée dans le dataset.")


dataset

In [ ]:
ideal_values = {
    "statistical_parity_difference": 0.0,
    "disparate_impact_ratio": 1.0,
    "equal_opportunity_difference": 0.0,
    "average_odds_difference": 0.0,
    "conditional_demographic_disparity": 0.0,
    "smoothed_edf": 1.0,
    "df_bias_amplification": 0.0
}
"""
def check_distance_to_ideal(metrics):
    dist = 0.0
    for metric in ideal_values.keys():
        dist += (ideal_values[metric]-metrics[metric])**2
    return np.sqrt(dist)

def calculate_metrics(csv_name):
    preddf = pd.read_csv("./expe_log/"+csv_name)
    preddf = convert_all_to_numericals(preddf)
    test_df = preddf[preddf["in_train"] == 0]

    
    preds = test_df["preds"]
    labels= test_df["labels"]
    weights = test_df["WEIGHTS"]


    metrics_after = get_group_metrics(
        y_true=labels,
        y_pred=preds,
        prot_attr=test_df[protected_attribute],
        priv_group=1,
        pos_label=1,
        sample_weight= weights if csv_name=="preds.csv" else None
    )

    
    # Affichage des résultats de fairness
    for metric, value in metrics_after.items():
        print(f"{metric}: {value:.4f}")

    print("Distance to ideal:",check_distance_to_ideal(metrics_after))

    return metrics_after,test_df
"""


In [ ]:
test_df = preddf[preddf["in_train"] == 0]
preds = test_df["preds"]
labels= test_df["labels"]

metrics_before = get_group_metrics(
    y_true=labels,
    y_pred=preds,
    prot_attr=test_dataset.protected_attributes[:, 0],
    priv_group=1,
    pos_label=0
)

# Affichage des résultats
for metric, value in metrics_before.items():
    print(f"{metric}: {value:.4f}")

In [ ]:
pred_rw = pd.read_csv("./expe_log/preds_RW.csv")

In [ ]:
def printMetrics(metric_orig,  metric_transf):
    print("\nMétriques avant ROC:")
    print("Equal Opportunity Diff:", metric_orig.equal_opportunity_difference())
    print("Disparate Impact:", metric_orig.disparate_impact())
    print("Average Odds Difference:", metric_orig.average_odds_difference())
    print("Theil Index:", metric_orig.theil_index())
    print("Statistical Parity Difference:", metric_orig.statistical_parity_difference())
    print("Error Rate:", metric_orig.error_rate())
    print("False Positive Rate:", metric_orig.false_positive_rate())
    print("False Negative Rate:", metric_orig.false_negative_rate())
    print("True Positive Rate:", metric_orig.true_positive_rate())
    print("True Negative Rate:", metric_orig.true_negative_rate())

    print("\nMétriques après ROC:")
    print("Equal Opportunity Diff:", metric_transf.equal_opportunity_difference())
    print("Disparate Impact:", metric_transf.disparate_impact())
    print("Average Odds Difference:", metric_transf.average_odds_difference())
    print("Theil Index:", metric_transf.theil_index())
    print("Statistical Parity Difference:", metric_transf.statistical_parity_difference())
    print("Error Rate:", metric_transf.error_rate())
    print("False Positive Rate:", metric_transf.false_positive_rate())
    print("False Negative Rate:", metric_transf.false_negative_rate())
    print("True Positive Rate:", metric_transf.true_positive_rate())
    print("True Negative Rate:", metric_transf.true_negative_rate())

### a. Reject Option Classification

In [ ]:
test_df = convert_all_to_numericals(test_df)

# Attribut protégé et groupes
protected_attribute = '+40ans'
privileged_groups = [{protected_attribute: 1}]
unprivileged_groups = [{protected_attribute: 0}]

# Créer les BinaryLabelDataset
test_dataset = BinaryLabelDataset(
    favorable_label=0,
    unfavorable_label=1,
    df=test_df.select_dtypes(include=['int64', 'float64']),
    label_names=["labels"],
    protected_attribute_names=[protected_attribute]
)

# Injecter prédictions et logits
test_with_preds = test_dataset.copy()
test_with_preds.labels = test_df["preds"].values.reshape(-1, 1)
test_with_preds.scores = test_df["logits_1"].values.reshape(-1, 1)

#### On va travailler sur le dataset des prédictions sans préprocessing

In [ ]:
def guided_ROC_search(
    test_dataset,
    test_with_preds,
    unprivileged_groups,
    privileged_groups,
    start_low=0.05,
    high=0.99,
    margin=10,
    step=0.01,
    max_iter=20,
    fairness_bounds=(-0.05, 0.05)
):
    current_low = start_low
    best_fn = float('inf')
    best_config = None
    history = []

    for i in range(max_iter):
        ROC = RejectOptionClassification(
            unprivileged_groups=unprivileged_groups,
            privileged_groups=privileged_groups,
            low_class_thresh=current_low,
            high_class_thresh=high,
            num_class_thresh=100,
            num_ROC_margin=margin,
            metric_name="Equal opportunity difference",
            metric_lb=fairness_bounds[0],
            metric_ub=fairness_bounds[1]
        )

        ROC = ROC.fit(test_dataset, test_with_preds)
        transformed = ROC.predict(test_with_preds)

        y_true = test_dataset.labels.ravel()
        y_pred = transformed.labels.ravel()

        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

        metric = ClassificationMetric(
            test_dataset,
            transformed,
            unprivileged_groups=unprivileged_groups,
            privileged_groups=privileged_groups
        )
        fairness_diff = metric.equal_opportunity_difference()

        history.append({
            'iteration': i,
            'low_thresh': round(current_low, 3),
            'FN': fn,
            'FP': fp,
            'TP': tp,
            'TN': tn,
            'fairness_diff': fairness_diff
        })

        # Garder le meilleur si fairness respectée
        if fairness_bounds[0] <= fairness_diff <= fairness_bounds[1]:
            if fn < best_fn:
                best_fn = fn
                best_config = {
                    'low_thresh': current_low,
                    'FN': fn,
                    'FP': fp,
                    'TP': tp,
                    'TN': tn,
                    'fairness_diff': fairness_diff
                }

        # Avancer quoi qu'il arrive
        current_low -= step
        if current_low < 0.0:
            break

    return best_config, history


In [ ]:
best_config, search_history = guided_ROC_search(
    test_dataset,
    test_with_preds,
    unprivileged_groups,
    privileged_groups,
    start_low=0.1,
    high=0.9,
    margin=50
)

print("Best config found:")
print(best_config)

import pandas as pd
print(pd.DataFrame(search_history))


In [ ]:
ROC = RejectOptionClassification(
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
    low_class_thresh=best_config['low_thresh'],
    high_class_thresh=0.99,
    num_class_thresh=100,
    num_ROC_margin=50,
    metric_name="Equal opportunity difference",
    metric_lb=-0.05,
    metric_ub=0.05
)


In [ ]:
ROC = ROC.fit(test_dataset, test_with_preds)
transformed_dataset = ROC.predict(test_with_preds)

In [ ]:
%matplotlib inline

# Matrice de confusion avant et après transformation
conf_matrix_orig = confusion_matrix(test_dataset.labels, test_with_preds.labels)
conf_matrix_transf = confusion_matrix(test_dataset.labels, transformed_dataset.labels)

# Normalisation en pourcentages
conf_matrix_orig_percentage = conf_matrix_orig / conf_matrix_orig.sum(axis=1)[:, np.newaxis] * 100
conf_matrix_transf_percentage = conf_matrix_transf / conf_matrix_transf.sum(axis=1)[:, np.newaxis] * 100

# Création des heatmaps
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Heatmap avant transformation (en pourcentages)
sns.heatmap(conf_matrix_orig_percentage, annot=True, fmt=".2f", cmap="Blues", 
            xticklabels=["Predicted Negative", "Predicted Positive"],
            yticklabels=["True Negative", "True Positive"], ax=axes[0], cbar_kws={'label': 'Percentage'})
axes[0].set_title('Matrice de Confusion Avant Transformation (en %)')
axes[0].set_xlabel('Prédictions')
axes[0].set_ylabel('Vraies étiquettes')

# Heatmap après transformation (en pourcentages)
sns.heatmap(conf_matrix_transf_percentage, annot=True, fmt=".2f", cmap="Blues", 
            xticklabels=["Predicted Negative", "Predicted Positive"],
            yticklabels=["True Negative", "True Positive"], ax=axes[1], cbar_kws={'label': 'Percentage'})
axes[1].set_title('Matrice de Confusion Après Transformation (en %)')
axes[1].set_xlabel('Prédictions')
axes[1].set_ylabel('Vraies étiquettes')

# Affichage
plt.tight_layout()
plt.show()


In [ ]:
# Calcul des métriques
metric_orig = ClassificationMetric(test_dataset, test_with_preds,
                                   unprivileged_groups=unprivileged_groups,
                                   privileged_groups=privileged_groups)

metric_transf = ClassificationMetric(test_dataset, transformed_dataset,
                                     unprivileged_groups=unprivileged_groups,
                                     privileged_groups=privileged_groups)


printMetrics(metric_orig,  metric_transf)


In [ ]:
%matplotlib inline
# Aplatir les tableaux au cas où
labels_before = test_with_preds.labels.ravel()
labels_after = transformed_dataset.labels.ravel()

# Tracer avec des histogrammes côte à côte
plt.figure(figsize=(6, 4))

sns.kdeplot(labels_before, color='blue', label='Avant ROC')
sns.kdeplot(labels_after, color='orange', label='Après ROC')


# Ajouter titres et légende propres
plt.title("Distribution des prédictions (Avant vs Après ROC)")
plt.xlabel("Label prédit (0 = sain, 1 = malade)")
plt.ylabel("Nombre d'occurrences")
plt.legend()
plt.xticks([0, 1])
plt.tight_layout()
plt.show()



#### Avec reweighing

In [ ]:
pred_rw = convert_all_to_numericals(pred_rw)

# Attribut protégé et groupes
protected_attribute = '+40ans'
privileged_groups = [{protected_attribute: 0}]
unprivileged_groups = [{protected_attribute: 1}]

# Créer les BinaryLabelDataset
test_dataset_rw = BinaryLabelDataset(
    favorable_label=0,
    unfavorable_label=1,
    df=pred_rw.select_dtypes(include=['int64', 'float64']),
    label_names=["labels"],
    protected_attribute_names=[protected_attribute]
)

# Injecter prédictions et logits
test_with_preds = test_dataset_rw.copy()
test_with_preds.labels = pred_rw["preds"].values.reshape(-1, 1)
test_with_preds.scores = pred_rw["logits_1"].values.reshape(-1, 1)

In [ ]:
best_config, search_history = guided_ROC_search(
    test_dataset_rw,
    test_with_preds,
    unprivileged_groups,
    privileged_groups,
    start_low=0.1,
    high=0.9,
)

print("Best config found:")
print(best_config)

import pandas as pd
print(pd.DataFrame(search_history))

In [ ]:
ROC = ROC.fit(test_dataset_rw, test_with_preds)
transformed_dataset = ROC.predict(test_with_preds)

In [ ]:
%matplotlib inline

# Matrice de confusion avant et après transformation
conf_matrix_orig = confusion_matrix(test_dataset_rw.labels, test_with_preds.labels)
conf_matrix_transf = confusion_matrix(test_dataset_rw.labels, transformed_dataset.labels)

# Normalisation en pourcentages
conf_matrix_orig_percentage = conf_matrix_orig / conf_matrix_orig.sum(axis=1)[:, np.newaxis] * 100
conf_matrix_transf_percentage = conf_matrix_transf / conf_matrix_transf.sum(axis=1)[:, np.newaxis] * 100

# Création des heatmaps
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Heatmap avant transformation (en pourcentages)
sns.heatmap(conf_matrix_orig_percentage, annot=True, fmt=".2f", cmap="Blues", 
            xticklabels=["Predicted Negative", "Predicted Positive"],
            yticklabels=["True Negative", "True Positive"], ax=axes[0], cbar_kws={'label': 'Percentage'})
axes[0].set_title('Matrice de Confusion Avant Transformation (en %)')
axes[0].set_xlabel('Prédictions')
axes[0].set_ylabel('Vraies étiquettes')

# Heatmap après transformation (en pourcentages)
sns.heatmap(conf_matrix_transf_percentage, annot=True, fmt=".2f", cmap="Blues", 
            xticklabels=["Predicted Negative", "Predicted Positive"],
            yticklabels=["True Negative", "True Positive"], ax=axes[1], cbar_kws={'label': 'Percentage'})
axes[1].set_title('Matrice de Confusion Après Transformation (en %)')
axes[1].set_xlabel('Prédictions')
axes[1].set_ylabel('Vraies étiquettes')

# Affichage
plt.tight_layout()
plt.show()


In [ ]:
# Calcul des métriques
metric_orig = ClassificationMetric(test_dataset_rw, test_with_preds,
                                   unprivileged_groups=unprivileged_groups,
                                   privileged_groups=privileged_groups)

metric_transf = ClassificationMetric(test_dataset_rw, transformed_dataset,
                                     unprivileged_groups=unprivileged_groups,
                                     privileged_groups=privileged_groups)

printMetrics(metric_orig,  metric_transf)  

In [ ]:
%matplotlib inline
# Aplatir les tableaux au cas où
labels_before = test_with_preds.labels.ravel()
labels_after = transformed_dataset.labels.ravel()

# Tracer avec des histogrammes côte à côte
plt.figure(figsize=(6, 4))

sns.kdeplot(labels_before, color='blue', label='Avant ROC')
sns.kdeplot(labels_after, color='orange', label='Après ROC')


# Ajouter titres et légende propres
plt.title("Distribution des prédictions (Avant vs Après ROC)")
plt.xlabel("Label prédit (0 = sain, 1 = malade)")
plt.ylabel("Nombre d'occurrences")
plt.legend()
plt.xticks([0, 1])
plt.tight_layout()
plt.show()

### b. Calibrated Equalized Odds

In [ ]:
def apply_CEO(test_dataset, priv_group=1, pos_label=0):
    cost_constraint = "fnr"

    # Postprocessing CEO
    ceo = CalibratedEqOddsPostprocessing(
        privileged_groups=[{protected_attribute: priv_group}],
        unprivileged_groups=[{protected_attribute: 1 - priv_group}],
        cost_constraint=cost_constraint,
        seed=42
    )

    # On considère test_dataset comme les prédictions (avec .scores et .labels modifiés)
    pred_dataset = test_dataset.copy()

    # Fit + Predict
    ceo = ceo.fit(test_dataset, pred_dataset)
    postproc_preds = ceo.predict(pred_dataset)

    # Calcul des métriques de fairness
    metrics = get_group_metrics(
        y_true=test_dataset.labels[:, 0],
        y_pred=postproc_preds.labels[:, 0],
        prot_attr=test_dataset.protected_attributes[:, 0],
        priv_group=priv_group,
        pos_label=pos_label
    )

    # Convertir les métriques en float
    metrics_float = {metric: float(value) for metric, value in metrics.items()}

    return metrics_float


In [ ]:
apply_CEO(test_dataset)

In [ ]:
apply_CEO(test_dataset_rw)

---

## V. Analyse et compréhension

### -> Get together and link preproc and postproc results

---

## VI. Conclusion

### *Preferably*, get together and discuss final comments